# Using invariant representations to guide future exploration

I would like to further explore an idea of leveraging abstract concepts learned from previous experiences to help aid and guide the behavior of an agent when faced with a novel problem. The aim is to sequentially construct and expand agent's knowledge base, and use it to construct a behavioral policy that would guide the exploration while learning the optimal policy.

## Implementation Plan

The following is a rough outline of the implementation plan:
- [ ] Allow training of agents via Q-Learning for wide range of tasks. At this stage, the goal is to setup the training framework, logging, metrics and everything you might need. In addition, make sure that the behavioral policy of the Q-Learning agent is swappable.
- [ ] Create invariant representation of learned experience. For each new task, first sample tuples of experiences, train an autoencoder network and add that particular task to the knowledge base.
- [ ] Create behavioral policy for a new task, from the knowledge base. At this stage, we alter the training process to:
  - [ ] Let the agent explore for a limited amount of time.
  - [ ] Determine a set of most relevant tasks.
  - [ ] Construct a behavior policy from a set of most relevant tasks and integrate it into Q-Learning.
- [ ] Lastly, I perform the experiment phase. Here, a carefully selected set of tasks  needs to be run, both individually as a baseline, and then in sequence using the knowledge base. If time permits, running several experiments with differently ordered tasks could be useful to obtain standard deviation.

In [3]:
import gym